In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
import torchvision
import os
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
import time
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
#from models import * #model and dataset
from train import *

if torch.cuda.is_available():  
    torch.cuda.empty_cache()
    device = "cuda:0" 
else:  
    device = "cpu" 

torch.manual_seed(16)
imgpath = 'Dataset/train_images'
dataset = pretrain_set(imgpath)
batch = 8

sz = len(dataset)
trlen = int(len(dataset)*0.8)
vllen = sz - trlen
trainset, validset = torch.utils.data.random_split(dataset,[trlen,vllen])

train_loader=DataLoader(trainset,batch,shuffle=True)
valid_loader = DataLoader(validset,batch,shuffle=True)

model = encoder().to(device)#remember to change output channel to 8
epochs = 5

optimizer = optim.Adadelta(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

for epoch in range(epochs):
    tg = []
    pd = []
    train_accuracy=0.0
    test_accuracy=0.0
    model.train()
    for data, target in tqdm(train_loader):
        data,target=data.to(device),target.to(device)
        optimizer.zero_grad()
        output=model(data.float())
        loss=loss_function(output,target)
        loss.backward()
        optimizer.step()
        _,prediction=torch.max(output.data,1)
        train_accuracy+=int(torch.sum(prediction==target.data))
    train_accuracy=train_accuracy/trlen 
    model.eval()
    
    for data, target in tqdm(valid_loader):
        data,target=data.to(device),target.to(device)
        output=model(data.float())
        _,prediction=torch.max(output.data,1)
        test_accuracy+=int(torch.sum(prediction==target.data))
        for k in prediction:
            pd.append(k.item())
        for l in target.data:
            tg.append(l.item())
    torch.save(model,'encoder.pt')
    test_accuracy=test_accuracy/vllen
    print('epoch '+str(epoch+1)+' train acurracy: '+str(train_accuracy)+' test accuracy: '+str(test_accuracy))
    print(confusion_matrix(tg,pd))